In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *


from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth

In [2]:
desde = '2022-04-27'
hasta = '2022-07-27'

In [3]:
query = """
WITH 
count_fulfillments_pasados AS (
	SELECT 
	    COUNT(DISTINCT boi.order_item_id) AS count_items,
	    bfg.order_id
     
	FROM postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"              bfg
	LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order"                     bo      ON bo.order_id = bfg.order_id
	LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"          ffg     ON bfg.fulfillment_group_id = ffg.fulfillment_group_id
	LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"         bfo     ON bfg.fulfillment_group_id = bfo.fulfillment_group_id
	LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment"             bop     ON bop.order_id = bo.order_id
	LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_order"                      fo      ON fo.order_id = bo.order_id
	INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON bo.site_disc = s.site_id
	LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"                fot     ON fot.fb_order_type_id=fo.fb_order_type_id
	INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"   bfgi    ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
	INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON bfgi.order_item_id= boi.order_item_id
	
	WHERE      
        ffg.close_date between '{desde}' AND '{hasta}'
        --AND s.site_identifier_value = {city}
        AND bo.order_status='SUBMITTED'
        AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
        AND fo.fb_order_status_id IN (1,6,7,8)
        AND bop.archived = 'N'
        AND (fot.name IS NULL OR fot.name <> 'REFUND')
	GROUP BY 2
),

ful AS (

    SELECT DISTINCT
        bo.order_id,
        ffg.close_date
        
    FROM postgres_broadleaf_federate."broadleaf.blc_order"                      bo
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"    bfg     ON bfg.order_id = bo.order_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"    bfo     ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"     ffg     ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                 fo      ON fo.order_id = bo.order_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment"        bop     ON bop.order_id = bo.order_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                 s       ON s.site_id = bo.site_disc
    LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"            fot     ON fot.fb_order_type_id=fo.fb_order_type_id
    LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_adjustment"     ba      ON ba.order_id = bo.order_id
    
    WHERE fo.fb_order_status_id IN (1,6,7,8)
        AND ffg.close_date between '{desde}' AND '{hasta}'
        --AND s.site_identifier_value = {city}
        AND bo.order_status = 'SUBMITTED'
        AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
        AND bop.archived = 'N'
        AND (fot.name IS NULL OR fot.name <> 'REFUND')
        
    GROUP BY 1,2
)

SELECT DISTINCT
    --bo.order_id,
    bo.customer_id,
    s.site_identifier_value as region_code,
    --TO_CHAR(ful.close_date, 'YYYY-mm-dd') AS day,
    bcat2.name as cat,
    CASE
        WHEN bcat.name ILIKE 'Pollo%' THEN 'Pollo'
        WHEN bcat.name ILIKE 'Res%' THEN 'Res'
        WHEN bcat.name ILIKE 'Cerdo%' THEN 'Cerdo'
        ELSE bcat.name
    END AS subcat,
    --bs.sku_id,
    --bs.addl_product_id AS padre_sku_id,
    --bs.name AS product,
    SUM(boi.quantity) AS quantity,
    CASE 
        WHEN s.site_identifier_value = 'CMX' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
        WHEN s.site_identifier_value = 'GDL' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
        WHEN s.site_identifier_value = 'PBC' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
        WHEN s.site_identifier_value = 'MTY' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
        WHEN s.site_identifier_value = 'SPO' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
        WHEN s.site_identifier_value = 'BHZ' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
        WHEN s.site_identifier_value = 'CWB' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
        WHEN s.site_identifier_value = 'VCP' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
    ELSE ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3000) END AS net_gmv_usd
    
FROM 
    postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"                  bfgi
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"            bfg     ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"             ffg     ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order"                        bo      ON bo.order_id = bfg.order_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                         s       ON bo.site_disc = s.site_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"                   boi     ON bfgi.order_item_id= boi.order_item_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                    foi     ON boi.order_item_id= foi.order_item_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_owner"                         fow     ON fow.owner_id = ffg.owner_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                         fo      ON fo.order_id = bo.order_id
    INNER JOIN ful                                                                              ON ful.order_id = bo.order_id
    LEFT JOIN count_fulfillments_pasados                                                cf      ON bfg.order_id = cf.order_id
    LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"                    fot     ON fot.fb_order_type_id=fo.fb_order_type_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"          bdoi    ON bdoi.order_item_id = boi.order_item_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs      ON bs.sku_id = bdoi.sku_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                      bp      ON bs.addl_product_id = bp.product_id
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"                 bcx     ON bcx.sub_category_id = bp.default_category_id and bcx.archived='N' and bcx.sndbx_tier is null and bcx.default_reference = 'true'
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"                      bcat    ON bp.default_category_id = bcat.category_id
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"                      bcat2   ON bcx.category_id = bcat2.category_id
    --LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"          boipd   ON boipd.order_item_id=boi.order_item_id
    --LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"            boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id
WHERE 
    ffg.close_date between '{desde}' AND '{hasta}'
    --AND s.site_identifier_value = {city}
    AND bo.order_status='SUBMITTED'
    AND fo.fb_order_status_id IN (1,6,7,8)
    AND (fot.name IS NULL OR fot.name <> 'REFUND')
    AND bcat2.name IN ('Abarrotes','Aseo e Higiene','Bebidas','Congelados','Desechables','Lácteos & Huevos','Carne, Pollo & Pescados', 'Frutas & Verduras', 'Bebidas','Mercearia','Limpeza e Higiene','Laticínios e Ovos','Frutas e Verduras','Congelados','Descartáveis','Carnes, Aves e Peixes')

GROUP BY 1,2,3,4--,5--,6--,7,8--,9
""".format(desde=desde, hasta=hasta, city='MTY')

df = read_connection_data_warehouse.runQuery(query)
df = df[(~df['region_code'].isin(['MTY']))]

In [4]:
#//----------------PRECARGA-------------------------//
#//Ruta de data // Base Data (Without Deleting) 
segmentos_full = get_fresh_query_result("https://internal-redash.federate.frubana.com/",101609,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [5]:
data_full = df.merge(segmentos_full[['customer_id','microsegment']], how = 'left', on = 'customer_id')
data_full['microsegment'] = data_full['microsegment'].fillna('Other')

In [6]:
#//Clean segment_names
cities_others = ['GDL', 'BHZ', 'PBC', 'MDE', 'VCP','CWB']

data_full.loc[data_full[data_full['region_code'].isin(cities_others)].index,('microsegment')] = 'Other'

data_full['microsegment'] = data_full['microsegment'].str.replace('s/café da manhã','')
data_full['microsegment'] = data_full['microsegment'].str.replace('c/café da manhã','')
data_full['microsegment'] = data_full['microsegment'].str.replace('c/desayuno','')
data_full['microsegment'] = data_full['microsegment'].str.replace('s/desayuno','')
data_full['microsegment'] = data_full['microsegment'].str.strip()
data_full['microsegment'] = data_full['microsegment'].apply(lambda x: 'Taquería' if 'Taquería' in x else x)
data_full['microsegment'] = data_full['microsegment'].apply(lambda x: x.replace('Parrilla/Asadero','Parrilla/asadero'))

data_full.loc[data_full[data_full['region_code'] == 'BOG'].index,('microsegment')] = data_full.loc[data_full[data_full['region_code'] == 'BOG'].index,('microsegment')].apply(lambda x: x.replace('Empanadas/Arepas','Empanadas/arepas'))
data_full.loc[data_full[data_full['region_code'] == 'BOG'].index,('microsegment')] = data_full.loc[data_full[data_full['region_code'] == 'BOG'].index,('microsegment')].apply(lambda x: x.replace('Fritos','Empanadas/arepas'))
data_full.loc[data_full[data_full['region_code'] == 'BOG'].index,('microsegment')] = data_full.loc[data_full[data_full['region_code'] == 'BOG'].index,('microsegment')].apply(lambda x: 'Panadería/Pastelería' if 'Panadería/Pastelería/Cafetería' in x else x)


data_full.loc[data_full[data_full['region_code'] == 'BAQ'].index,('microsegment')] = data_full.loc[data_full[data_full['region_code'] == 'BAQ'].index,('microsegment')].apply(lambda x: 'Panadería/Pastelería/Cafetería' if 'Cafetería' in x else x)
data_full.loc[data_full[data_full['region_code'] == 'BAQ'].index,('microsegment')] = data_full.loc[data_full[data_full['region_code'] == 'BAQ'].index,('microsegment')].apply(lambda x: 'Panadería/Pastelería/Cafetería' if 'Panadería/Pastelería' in x else x)
data_full.loc[data_full[data_full['region_code'] == 'BAQ'].index,('microsegment')] = data_full.loc[data_full[data_full['region_code'] == 'BAQ'].index,('microsegment')].apply(lambda x: x.replace('Empanadas/Arepas','Fritos'))

data_full.loc[data_full[data_full['region_code'] == 'BOG'].index,('microsegment')] = data_full.loc[data_full[data_full['region_code'] == 'BOG'].index,('microsegment')].apply(lambda x: x.replace('Empanadas/Arepas','Empanadas/arepas'))

In [7]:
data_full['subcat'] = data_full['subcat'].str.strip()
data_full.dropna(axis=0, subset=['customer_id'], inplace=True)
data_full['customer_id'] = data_full['customer_id'].astype('str')

In [8]:
def cross_sell(city, microsegment):
    """
    This function aim to find the relation between subcategories to increase the probability of buying both at the same time (cross_sell) 
    and its filtered by a segment of clients to aim a better understanding of the group

    Args:
        city (str): The city that you want to figure out its cross_sell of subcats
        microsegment (str): The microsegment of the city

    Returns:
        subcats: A list with the subcategories that generate the best mix for the cross_sell
        df: A data frame with the info of the  antecedents and consequents and their confidence values
    """
    
    basket = (data_full[(data_full['region_code'] == city) & (data_full['microsegment'] == microsegment)]
          .groupby(['customer_id', 'subcat'])['quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('customer_id'))
    
    def encode_units(x):
        if x <= 0:
            return 0
        if x >= 1:
            return 1

    basket_sets = basket.applymap(encode_units)
    
    frequent_itemsets = fpgrowth(basket_sets.astype(bool), min_support=0.10, use_colnames=True)
    frequent_itemsets
    
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.70) 
    #support how many times of be together (P(AuC)) 
    #confidence is equal to P(C/A)
    #lift is to evaluate the support and confidence rull, there are times when both of these measures may be high, and yet still produce a rule that is not useful -> (Rule Support) /(Support(Antecedent) * Support(Consequent))
    """
    Any rule with an improvement of less than 1 does not indicate a real cross-selling opportunity,
    no matter how high its support and confidence, because it actually offers less ability to predict a purchase than does random chance.
    """
    antecedents = list(map(list, rules.antecedents.values))  
    antecedents1 = set([x for xs in antecedents for x in xs])

    consequents = list(map(list, rules.consequents.values))  
    consequents1 = set([x for xs in consequents for x in xs])

    set3 = antecedents1.union(consequents1)
    
    return list(set3), rules.sort_values(by = 'confidence', ascending=False)
    

In [9]:
city= 'BOG'
data_full[['region_code', 'microsegment']][(data_full['region_code'] == city)].drop_duplicates().values[:,1]

array(['Corrientazo', 'Panadería/Pastelería', 'Comida especializada',
       'Empanadas/arepas', 'Comida rápida', 'Parrilla/asadero', 'Other',
       'Cafetería'], dtype=object)

In [10]:
parametros = {}
d = {}
for i in data_full['region_code'].unique():
    parametros[i] = {}
    for j in data_full[['region_code', 'microsegment']][(data_full['region_code'] == i)].drop_duplicates().values[:,1]:
        parametros[i][j], d["df_xsell_{city}_{ms}".format(city=i,ms=j)] = cross_sell(i, j)

In [17]:
#print(parametros)

In [16]:
parametros_2 = {
'SPO': 
    {'Comida internacional/Grill':
        ['Condimentos', 'Papéis', 'Ovos', 'Verduras', 'Laticínios', 'Legumes', 'Outros Limpadores', 'Frango', 'Tubérculos', 'Cervejas', 'Café, Chocolates e Infusões', 'Batatas Pré-Fritas', 'Temperos', 'Refrigerantes e Energéticos', 'Farinhas e Misturas', 'Massas e Molhos', 'Utensílios de Limpeza', 'Queijos', 'Carne Vermelha', 'Sucos e Chás', 'Feijão', 'Açúcar, Adoçantes e Doces', 'Detergente, Sabão e Lava-Louças', 'Polpas e Açaí', 'Frutas', 'Manteigas e Margarinas', 'Suínos', 'Embutidos', 'Águas e Isotônicos', 'Alimentos Congelados', 'Azeites, Óleos e Vinagres', 'Arroz', 'Conservas e Enlatados', 'Leite'], 
    'Comida por quilo':
        ['Condimentos', 'Papéis', 'Laticínios', 'Verduras', 'Ovos', 'Legumes', 'Outros Limpadores', 'Frango', 'Tubérculos', 'Cervejas', 'Café, Chocolates e Infusões', 'Batatas Pré-Fritas', 'Temperos', 'Refrigerantes e Energéticos', 'Utensílios', 'Massas e Molhos', 'Farinhas e Misturas', 'Utensílios de Limpeza', 'Queijos', 'Carne Vermelha', 'Sucos e Chás', 'Feijão', 'Açúcar, Adoçantes e Doces', 'Detergente, Sabão e Lava-Louças', 'Polpas e Açaí', 'Frutas', 'Manteigas e Margarinas', 'Suínos', 'Embutidos', 'Águas e Isotônicos', 'Alimentos Congelados', 'Azeites, Óleos e Vinagres', 'Arroz', 'Conservas e Enlatados', 'Leite'], 
    'Sucos/Açaí':
        ['Condimentos', 'Papéis', 'Laticínios', 'Verduras', 'Legumes', 'Outros Limpadores', 'Frango', 'Tubérculos', 'Cervejas', 'Café, Chocolates e Infusões', 'Batatas Pré-Fritas', 'Refrigerantes e Energéticos', 'Temperos', 'Farinhas e Misturas', 'Massas e Molhos', 'Utensílios de Limpeza', 'Queijos', 'Carne Vermelha', 'Sucos e Chás', 'Feijão', 'Açúcar, Adoçantes e Doces', 'Detergente, Sabão e Lava-Louças', 'Polpas e Açaí', 'Frutas', 'Suínos', 'Manteigas e Margarinas', 'Embutidos', 'Águas e Isotônicos', 'Alimentos Congelados', 'Azeites, Óleos e Vinagres', 'Arroz', 'Leite'], 
    'Padaria/Cafeteria/Bolos':
        ['Condimentos', 'Papéis', 'Laticínios', 'Verduras', 'Ovos', 'Legumes', 'Outros Limpadores', 'Frango', 'Tubérculos', 'Cervejas', 'Café, Chocolates e Infusões', 'Batatas Pré-Fritas', 'Refrigerantes e Energéticos', 'Temperos', 'Farinhas e Misturas', 'Massas e Molhos', 'Utensílios de Limpeza', 'Queijos', 'Carne Vermelha', 'Sucos e Chás', 'Feijão', 'Açúcar, Adoçantes e Doces', 'Detergente, Sabão e Lava-Louças', 'Polpas e Açaí', 'Frutas', 'Manteigas e Margarinas', 'Embutidos', 'Águas e Isotônicos', 'Alimentos Congelados', 'Azeites, Óleos e Vinagres', 'Arroz', 'Conservas e Enlatados', 'Leite'], 
    'Lanches':
        ['Condimentos', 'Papéis', 'Laticínios', 'Verduras', 'Ovos', 'Legumes', 'Outros Limpadores', 'Frango', 'Tubérculos', 'Cervejas', 'Café, Chocolates e Infusões', 'Batatas Pré-Fritas', 'Temperos', 'Refrigerantes e Energéticos', 'Massas e Molhos', 'Farinhas e Misturas', 'Utensílios de Limpeza', 'Queijos', 'Carne Vermelha', 'Sucos e Chás', 'Feijão', 'Açúcar, Adoçantes e Doces', 'Detergente, Sabão e Lava-Louças', 'Polpas e Açaí', 'Frutas', 'Suínos', 'Manteigas e Margarinas', 'Embutidos', 'Águas e Isotônicos', 'Alimentos Congelados', 'Azeites, Óleos e Vinagres', 'Arroz', 'Conservas e Enlatados', 'Leite'], 
    'Prato do dia':
        ['Condimentos', 'Papéis', 'Laticínios', 'Verduras', 'Ovos', 'Legumes', 'Outros Limpadores', 'Frango', 'Tubérculos', 'Cervejas', 'Café, Chocolates e Infusões', 'Batatas Pré-Fritas', 'Temperos', 'Refrigerantes e Energéticos', 'Farinhas e Misturas', 'Massas e Molhos', 'Utensílios de Limpeza', 'Queijos', 'Carne Vermelha', 'Sucos e Chás', 'Feijão', 'Açúcar, Adoçantes e Doces', 'Detergente, Sabão e Lava-Louças', 'Polpas e Açaí', 'Frutas', 'Suínos', 'Manteigas e Margarinas', 'Embutidos', 'Águas e Isotônicos', 'Alimentos Congelados', 'Azeites, Óleos e Vinagres', 'Arroz', 'Conservas e Enlatados', 'Leite'], 
    'Comida brasileira y saudavel':
        ['Condimentos', 'Papéis', 'Ovos', 'Verduras', 'Laticínios', 'Legumes', 'Outros Limpadores', 'Frango', 'Tubérculos', 'Cervejas', 'Café, Chocolates e Infusões', 'Batatas Pré-Fritas', 'Temperos', 'Refrigerantes e Energéticos', 'Farinhas e Misturas', 'Massas e Molhos', 'Utensílios de Limpeza', 'Queijos', 'Carne Vermelha', 'Sucos e Chás', 'Feijão', 'Açúcar, Adoçantes e Doces', 'Detergente, Sabão e Lava-Louças', 'Polpas e Açaí', 'Frutas', 'Suínos', 'Manteigas e Margarinas', 'Embutidos', 'Águas e Isotônicos', 'Alimentos Congelados', 'Azeites, Óleos e Vinagres', 'Arroz', 'Conservas e Enlatados', 'Leite'], 
    'Other':
        ['Condimentos', 'Papéis', 'Laticínios', 'Verduras', 'Ovos', 'Legumes', 'Outros Limpadores', 'Frango', 'Tubérculos', 'Cervejas', 'Café, Chocolates e Infusões', 'Batatas Pré-Fritas', 'Temperos', 'Refrigerantes e Energéticos', 'Farinhas e Misturas', 'Massas e Molhos', 'Utensílios de Limpeza', 'Queijos', 'Carne Vermelha', 'Sucos e Chás', 'Feijão', 'Açúcar, Adoçantes e Doces', 'Detergente, Sabão e Lava-Louças', 'Frutas', 'Suínos', 'Manteigas e Margarinas', 'Embutidos', 'Águas e Isotônicos', 'Azeites, Óleos e Vinagres', 'Arroz', 'Conservas e Enlatados', 'Leite'], 
    'Pizza/Comida Italiana':
        ['Condimentos', 'Papéis', 'Ovos', 'Verduras', 'Laticínios', 'Legumes', 'Outros Limpadores', 'Tubérculos', 'Frango', 'Cervejas', 'Café, Chocolates e Infusões', 'Batatas Pré-Fritas', 'Temperos', 'Refrigerantes e Energéticos', 'Massas e Molhos', 'Farinhas e Misturas', 'Utensílios de Limpeza', 'Queijos', 'Carne Vermelha', 'Sucos e Chás', 'Feijão', 'Açúcar, Adoçantes e Doces', 'Detergente, Sabão e Lava-Louças', 'Frutas', 'Manteigas e Margarinas', 'Suínos', 'Embutidos', 'Águas e Isotônicos', 'Alimentos Congelados', 'Azeites, Óleos e Vinagres', 'Arroz', 'Conservas e Enlatados', 'Leite']
    }, 
'PBC': 
    {'Other':
        ['Leches', 'Enlatados', 'Pollo', 'Verduras', 'Refrescos', 'Tubérculos', 'Pasta', 'Papel & Toallas', 'Detergente, Jabón & Lavatrastes', 'Frutas', 'Jugos', 'Cerdo', 'Aceites & Grasas', 'Azúcar & Endulzantes', 'Harinas & Mezclas', 'Embutidos', 'Alimentos Congelados', 'Granos', 'Salsas', 'Arroz', 'Derivados Lácteos', 'Vasos, Contenedores & Platos']
    }, 
'CMX': 
    {'Other':
        ['Chiles Secos', 'Leches', 'Enlatados', 'Panadería & Tortilleria', 'Sal & Sazonadores', 'Pollo', 'Verduras', 'Cremas', 'Refrescos', 'Tubérculos', 'Papel & Toallas', 'Pasta', 'Detergente, Jabón & Lavatrastes', 'Res', 'Quesos', 'Frutas', 'Implementos de Aseo', 'Cerdo', 'Aceites & Grasas', 'Azúcar & Endulzantes', 'Harinas & Mezclas', 'Embutidos', 'Salsas', 'Granos', 'Arroz', 'Derivados Lácteos', 'Desinfectantes & Sanitizantes', 'Vasos, Contenedores & Platos'], 
    'Panadería/Pastelería':
        ['Leches', 'Mantequillas & Margarinas', 'Enlatados', 'Verduras', 'Tubérculos', 'Papel & Toallas', 'Pasta', 'Detergente, Jabón & Lavatrastes', 'Café, Chocolate e Infusiones', 'Frutas', 'Implementos de Aseo', 'Huevos', 'Harinas & Mezclas', 'Azúcar & Endulzantes', 'Aceites & Grasas', 'Granos', 'Arroz', 'Derivados Lácteos', 'Desinfectantes & Sanitizantes', 'Vasos, Contenedores & Platos'], 
    'Fonda':
        ['Chiles Secos', 'Panadería & Tortilleria', 'Pollo', 'Sal & Sazonadores', 'Verduras', 'Cremas', 'Refrescos', 'Tubérculos', 'Papel & Toallas', 'Pasta', 'Café, Chocolate e Infusiones', 'Huevos', 'Cerdo', 'Aceites & Grasas', 'Azúcar & Endulzantes', 'Harinas & Mezclas', 'Granos', 'Derivados Lácteos', 'Leches', 'Enlatados', 'Detergente, Jabón & Lavatrastes', 'Res', 'Quesos', 'Frutas', 'Implementos de Aseo', 'Embutidos', 'Salsas', 'Arroz', 'Desinfectantes & Sanitizantes', 'Vasos, Contenedores & Platos'], 
    'Comida rápida':
        ['Chiles Secos', 'Leches', 'Enlatados', 'Panadería & Tortilleria', 'Sal & Sazonadores', 'Pollo', 'Verduras', 'Refrescos', 'Cremas', 'Tubérculos', 'Papel & Toallas', 'Pasta', 'Detergente, Jabón & Lavatrastes', 'Res', 'Quesos', 'Frutas', 'Jugos', 'Implementos de Aseo', 'Cerdo', 'Aceites & Grasas', 'Azúcar & Endulzantes', 'Harinas & Mezclas', 'Embutidos', 'Salsas', 'Granos', 'Alimentos Congelados', 'Arroz', 'Derivados Lácteos', 'Desinfectantes & Sanitizantes', 'Vasos, Contenedores & Platos'], 
    'Taquería':
        ['Chiles Secos', 'Leches', 'Enlatados', 'Panadería & Tortilleria', 'Sal & Sazonadores', 'Pollo', 'Verduras', 'Refrescos', 'Cremas', 'Tubérculos', 'Pasta', 'Papel & Toallas', 'Detergente, Jabón & Lavatrastes', 'Res', 'Quesos', 'Frutas', 'Jugos', 'Implementos de Aseo', 'Cerdo', 'Aceites & Grasas', 'Azúcar & Endulzantes', 'Harinas & Mezclas', 'Embutidos', 'Salsas', 'Granos', 'Arroz', 'Derivados Lácteos', 'Desinfectantes & Sanitizantes', 'Vasos, Contenedores & Platos'], 
    'Cafetería':
        ['Chiles Secos', 'Panadería & Tortilleria', 'Pollo', 'Sal & Sazonadores', 'Verduras', 'Cremas', 'Refrescos', 'Tubérculos', 'Pasta', 'Papel & Toallas', 'Café, Chocolate e Infusiones', 'Huevos', 'Aceites & Grasas', 'Azúcar & Endulzantes', 'Harinas & Mezclas', 'Granos', 'Derivados Lácteos', 'Leches', 'Enlatados', 'Detergente, Jabón & Lavatrastes', 'Quesos', 'Res', 'Frutas', 'Implementos de Aseo', 'Jugos', 'Embutidos', 'Salsas', 'Arroz', 'Desinfectantes & Sanitizantes', 'Vasos, Contenedores & Platos'], 
    'Juguería/Comida saludable':
        ['Leches', 'Enlatados', 'Pollo', 'Verduras', 'Refrescos', 'Tubérculos', 'Pasta', 'Papel & Toallas', 'Detergente, Jabón & Lavatrastes', 'Frutas', 'Implementos de Aseo', 'Aceites & Grasas', 'Azúcar & Endulzantes', 'Salsas', 'Granos', 'Arroz', 'Derivados Lácteos', 'Desinfectantes & Sanitizantes', 'Vasos, Contenedores & Platos'], 
    'Comida especializada':
        ['Chiles Secos', 'Panadería & Tortilleria', 'Sal & Sazonadores', 'Pollo', 'Pescados & Mariscos Congelado', 'Verduras', 'Refrescos', 'Cremas', 'Tubérculos', 'Papel & Toallas', 'Pasta', 'Cerdo', 'Aceites & Grasas', 'Azúcar & Endulzantes', 'Harinas & Mezclas', 'Granos', 'Derivados Lácteos', 'Leches', 'Enlatados', 'Detergente, Jabón & Lavatrastes', 'Res', 'Quesos', 'Frutas', 'Implementos de Aseo', 'Jugos', 'Embutidos', 'Salsas', 'Arroz', 'Desinfectantes & Sanitizantes', 'Vasos, Contenedores & Platos']
    }, 
'BOG': 
    {'Corrientazo':
        ['Pollo', 'Bolsas', 'Sal & Sazonadores', 'Verduras', 'Tubérculos', 'Papel & Toallas', 'Pasta', 'Huevos', 'Cerdo', 'Aceites & Grasas', 'Contenedores', 'Cubiertos & Pitillos', 'Azúcar & Endulzantes', 'Detergente, Jabón & Lavalozas', 'Harinas & Mezclas', 'Granos', 'Frutas Jugo', 'Leches', 'Vasos', 'Res', 'Frutas', 'Implementos de Aseo', 'Gaseosas', 'Salsas', 'Alimentos Congelados', 'Arroz', 'Desinfectantes & Sanitizantes'], 
    'Panadería/Pastelería':
        ['Leches', 'Bolsas', 'Pollo', 'Verduras', 'Vasos', 'Tubérculos', 'Papel & Toallas', 'Platos & Bandejas', 'Res', 'Café, Chocolate e Infusiones', 'Frutas', 'Implementos de Aseo', 'Huevos', 'Cerdo', 'Harinas & Mezclas', 'Azúcar & Endulzantes', 'Aceites & Grasas', 'Cubiertos & Pitillos', 'Detergente, Jabón & Lavalozas', 'Contenedores', 'Gaseosas', 'Frutas Jugo', 'Arroz', 'Desinfectantes & Sanitizantes'], 
    'Comida especializada':
        ['Leches', 'Pollo', 'Bolsas', 'Sal & Sazonadores', 'Verduras', 'Vasos', 'Tubérculos', 'Papel & Toallas', 'Pulpas de Fruta', 'Pasta', 'Res', 'Frutas', 'Implementos de Aseo', 'Huevos', 'Cerdo', 'Aceites & Grasas', 'Contenedores', 'Azúcar & Endulzantes', 'Cubiertos & Pitillos', 'Detergente, Jabón & Lavalozas', 'Harinas & Mezclas', 'Gaseosas', 'Granos', 'Alimentos Congelados', 'Salsas', 'Arroz', 'Desinfectantes & Sanitizantes'], 
    'Empanadas/arepas':
        ['Leches', 'Pollo', 'Sal & Sazonadores', 'Bolsas', 'Verduras', 'Vasos', 'Tubérculos', 'Papel & Toallas', 'Res', 'Café, Chocolate e Infusiones', 'Frutas', 'Implementos de Aseo', 'Huevos', 'Cerdo', 'Aceites & Grasas', 'Harinas & Mezclas', 'Cubiertos & Pitillos', 'Azúcar & Endulzantes', 'Detergente, Jabón & Lavalozas', 'Contenedores', 'Embutidos', 'Granos', 'Gaseosas', 'Salsas', 'Arroz', 'Desinfectantes & Sanitizantes'], 
    'Comida rápida':
        ['Leches', 'Pollo', 'Bolsas', 'Sal & Sazonadores', 'Arroz', 'Verduras', 'Vasos', 'Tubérculos', 'Papel & Toallas', 'Res', 'Frutas', 'Implementos de Aseo', 'Huevos', 'Cerdo', 'Aceites & Grasas', 'Contenedores', 'Harinas & Mezclas', 'Cubiertos & Pitillos', 'Detergente, Jabón & Lavalozas', 'Azúcar & Endulzantes', 'Embutidos', 'Granos', 'Gaseosas', 'Salsas', 'Alimentos Congelados', 'Desinfectantes & Sanitizantes'], 
    'Parrilla/asadero':
        ['Leches', 'Sal & Sazonadores', 'Pollo', 'Bolsas', 'Verduras', 'Vasos', 'Tubérculos', 'Papel & Toallas', 'Res', 'Frutas', 'Implementos de Aseo', 'Huevos', 'Cerdo', 'Aceites & Grasas', 'Contenedores', 'Harinas & Mezclas', 'Cubiertos & Pitillos', 'Detergente, Jabón & Lavalozas', 'Gaseosas', 'Salsas', 'Granos', 'Alimentos Congelados', 'Arroz', 'Desinfectantes & Sanitizantes'], 
    'Other':
        ['Leches', 'Pollo', 'Sal & Sazonadores', 'Bolsas', 'Arroz', 'Verduras', 'Vasos', 'Tubérculos', 'Papel & Toallas', 'Pulpas de Fruta', 'Res', 'Frutas', 'Implementos de Aseo', 'Huevos', 'Cerdo', 'Aceites & Grasas', 'Azúcar & Endulzantes', 'Harinas & Mezclas', 'Contenedores', 'Detergente, Jabón & Lavalozas', 'Cubiertos & Pitillos', 'Gaseosas', 'Granos', 'Frutas Jugo', 'Salsas', 'Alimentos Congelados', 'Desinfectantes & Sanitizantes'], 
    'Cafetería':
        ['Leches', 'Pollo', 'Sal & Sazonadores', 'Bolsas', 'Verduras', 'Vasos', 'Tubérculos', 'Papel & Toallas', 'Pulpas de Fruta', 'Res', 'Café, Chocolate e Infusiones', 'Frutas', 'Implementos de Aseo', 'Huevos', 'Cerdo', 'Aceites & Grasas', 'Azúcar & Endulzantes', 'Cubiertos & Pitillos', 'Harinas & Mezclas', 'Detergente, Jabón & Lavalozas', 'Contenedores', 'Embutidos', 'Granos', 'Frutas Jugo', 'Gaseosas', 'Arroz', 'Desinfectantes & Sanitizantes']
    }, 
'BHZ': 
    {'Other':
        ['Condimentos', 'Papéis', 'Laticínios', 'Verduras', 'Ovos', 'Legumes', 'Outros Limpadores', 'Frango', 'Tubérculos', 'Cervejas', 'Café, Chocolates e Infusões', 'Batatas Pré-Fritas', 'Refrigerantes e Energéticos', 'Temperos', 'Farinhas e Misturas', 'Massas e Molhos', 'Utensílios de Limpeza', 'Carne Vermelha', 'Sucos e Chás', 'Feijão', 'Açúcar, Adoçantes e Doces', 'Detergente, Sabão e Lava-Louças', 'Frutas', 'Embutidos', 'Águas e Isotônicos', 'Azeites, Óleos e Vinagres', 'Arroz', 'Conservas e Enlatados', 'Leite']
    }, 
'BAQ': 
    {'Corrientazo':
        ['Leches', 'Pollo', 'Sal & Sazonadores', 'Bolsas', 'Pescados & Mariscos Congelado', 'Verduras', 'Vasos', 'Cremas', 'Tubérculos', 'Papel & Toallas', 'Pasta', 'Res', 'Frutas', 'Implementos de Aseo', 'Huevos', 'Cerdo', 'Aceites & Grasas', 'Azúcar & Endulzantes', 'Contenedores', 'Cubiertos & Pitillos', 'Detergente, Jabón & Lavalozas', 'Harinas & Mezclas', 'Embutidos', 'Granos', 'Gaseosas', 'Alimentos Congelados', 'Arroz', 'Desinfectantes & Sanitizantes'], 
    'Comida especializada':
        ['Leches', 'Pollo', 'Bolsas', 'Pescados & Mariscos Congelado', 'Verduras', 'Vasos', 'Cremas', 'Platos & Bandejas', 'Tubérculos', 'Papel & Toallas', 'Res', 'Frutas', 'Huevos', 'Cerdo', 'Aceites & Grasas', 'Azúcar & Endulzantes', 'Cubiertos & Pitillos', 'Contenedores', 'Detergente, Jabón & Lavalozas', 'Harinas & Mezclas', 'Embutidos', 'Gaseosas', 'Arroz', 'Desinfectantes & Sanitizantes'], 
    'Other':
        ['Leches', 'Pollo', 'Bolsas', 'Verduras', 'Vasos', 'Tubérculos', 'Pasta', 'Papel & Toallas', 'Res', 'Frutas', 'Implementos de Aseo', 'Huevos', 'Cerdo', 'Aceites & Grasas', 'Azúcar & Endulzantes', 'Cubiertos & Pitillos', 'Contenedores', 'Detergente, Jabón & Lavalozas', 'Harinas & Mezclas', 'Embutidos', 'Granos', 'Gaseosas', 'Alimentos Congelados', 'Arroz', 'Desinfectantes & Sanitizantes'], 
    'Panadería/Pastelería/Cafetería':
        ['Pollo', 'Bolsas', 'Verduras', 'Cremas', 'Panadería & Repostería', 'Platos & Bandejas', 'Tubérculos', 'Papel & Toallas', 'Huevos', 'Cerdo', 'Aceites & Grasas', 'Azúcar & Endulzantes', 'Harinas & Mezclas', 'Contenedores', 'Detergente, Jabón & Lavalozas', 'Cubiertos & Pitillos', 'Granos', 'Leches', 'Vasos', 'Res', 'Frutas', 'Implementos de Aseo', 'Embutidos', 'Gaseosas', 'Arroz', 'Desinfectantes & Sanitizantes'], 
    'Comida rápida':
        ['Pollo', 'Bolsas', 'Verduras', 'Cremas', 'Tubérculos', 'Papel & Toallas', 'Platos & Bandejas', 'Huevos', 'Cerdo', 'Aceites & Grasas', 'Contenedores', 'Cubiertos & Pitillos', 'Harinas & Mezclas', 'Detergente, Jabón & Lavalozas', 'Azúcar & Endulzantes', 'Granos', 'Leches', 'Vasos', 'Res', 'Quesos', 'Frutas', 'Implementos de Aseo', 'Embutidos', 'Gaseosas', 'Alimentos Congelados', 'Salsas', 'Arroz', 'Desinfectantes & Sanitizantes'], 
    'Fritos':
        ['Leches', 'Pollo', 'Bolsas', 'Verduras', 'Vasos', 'Cremas', 'Tubérculos', 'Papel & Toallas', 'Res', 'Frutas', 'Implementos de Aseo', 'Huevos', 'Cerdo', 'Aceites & Grasas', 'Azúcar & Endulzantes', 'Cubiertos & Pitillos', 'Harinas & Mezclas', 'Detergente, Jabón & Lavalozas', 'Contenedores', 'Embutidos', 'Granos', 'Gaseosas', 'Arroz', 'Desinfectantes & Sanitizantes'], 
    'Comida de mar y Colombiana':
        ['Pollo', 'Bolsas', 'Sal & Sazonadores', 'Verduras', 'Cremas', 'Tubérculos', 'Papel & Toallas', 'Huevos', 'Cerdo', 'Aceites & Grasas', 'Contenedores', 'Azúcar & Endulzantes', 'Harinas & Mezclas', 'Detergente, Jabón & Lavalozas', 'Cubiertos & Pitillos', 'Granos', 'Leches', 'Vasos', 'Res', 'Frutas', 'Implementos de Aseo', 'Embutidos', 'Gaseosas', 'Alimentos Congelados', 'Arroz', 'Desinfectantes & Sanitizantes'], 
    'Parrilla/asadero':
        ['Pollo', 'Bolsas', 'Sal & Sazonadores', 'Verduras', 'Cremas', 'Tubérculos', 'Papel & Toallas', 'Pasta', 'Huevos', 'Cerdo', 'Aceites & Grasas', 'Contenedores', 'Azúcar & Endulzantes', 'Cubiertos & Pitillos', 'Detergente, Jabón & Lavalozas', 'Harinas & Mezclas', 'Granos', 'Leches', 'Vasos', 'Res', 'Frutas', 'Implementos de Aseo', 'Embutidos', 'Gaseosas', 'Alimentos Congelados', 'Salsas', 'Arroz', 'Desinfectantes & Sanitizantes'], 
    'Tienda/Restaurante':
        ['Pollo', 'Bolsas', 'Verduras', 'Tubérculos', 'Papel & Toallas', 'Huevos', 'Cerdo', 'Aceites & Grasas', 'Cubiertos & Pitillos', 'Contenedores', 'Azúcar & Endulzantes', 'Detergente, Jabón & Lavalozas', 'Granos', 'Leches', 'Vasos', 'Res', 'Frutas', 'Implementos de Aseo', 'Embutidos', 'Gaseosas', 'Alimentos Congelados', 'Arroz', 'Desinfectantes & Sanitizantes']
    }, 
'GDL': 
    {'Other':
        ['Leches', 'Enlatados', 'Pollo', 'Verduras', 'Refrescos', 'Tubérculos', 'Papel & Toallas', 'Pasta', 'Detergente, Jabón & Lavatrastes', 'Res', 'Cubiertos', 'Frutas', 'Jugos', 'Cerdo', 'Aceites & Grasas', 'Azúcar & Endulzantes', 'Harinas & Mezclas', 'Embutidos', 'Salsas', 'Granos', 'Alimentos Congelados', 'Arroz', 'Derivados Lácteos', 'Desinfectantes & Sanitizantes', 'Vasos, Contenedores & Platos']
    }, 
'CWB': 
    {'Other':
        ['Condimentos', 'Papéis', 'Laticínios', 'Verduras', 'Ovos', 'Legumes', 'Outros Limpadores', 'Frango', 'Tubérculos', 'Cervejas', 'Café, Chocolates e Infusões', 'Batatas Pré-Fritas', 'Refrigerantes e Energéticos', 'Temperos', 'Farinhas e Misturas', 'Massas e Molhos', 'Utensílios de Limpeza', 'Sucos e Chás', 'Feijão', 'Hortaliças', 'Açúcar, Adoçantes e Doces', 'Detergente, Sabão e Lava-Louças', 'Frutas', 'Manteigas e Margarinas', 'Embutidos', 'Águas e Isotônicos', 'Alimentos Congelados', 'Azeites, Óleos e Vinagres', 'Arroz', 'Leite']
    }, 
'VCP': 
    {'Other':
        ['Sucos e Chás', 'Papéis', 'Refrigerantes e Energéticos', 'Laticínios', 'Verduras', 'Ovos', 'Legumes', 'Outros Limpadores', 'Feijão', 'Tubérculos', 'Açúcar, Adoçantes e Doces', 'Cervejas', 'Detergente, Sabão e Lava-Louças', 'Massas e Molhos', 'Frutas', 'Café, Chocolates e Infusões', 'Águas e Isotônicos', 'Azeites, Óleos e Vinagres', 'Temperos', 'Arroz', 'Conservas e Enlatados', 'Farinhas e Misturas', 'Leite', 'Utensílios de Limpeza']
    }, 
'MDE': 
    {'Other':
        ['Leches', 'Bolsas', 'Sal & Sazonadores', 'Verduras', 'Vasos', 'Tubérculos', 'Papel & Toallas', 'Café, Chocolate e Infusiones', 'Frutas', 'Huevos', 'Aceites & Grasas', 'Azúcar & Endulzantes', 'Harinas & Mezclas', 'Contenedores', 'Detergente, Jabón & Lavalozas', 'Cubiertos & Pitillos', 'Gaseosas', 'Granos', 'Cervezas & Licores', 'Arroz', 'Desinfectantes & Sanitizantes']
    }
}


In [12]:
d.keys()

dict_keys(['df_xsell_SPO_Comida internacional/Grill', 'df_xsell_SPO_Comida por quilo', 'df_xsell_SPO_Sucos/Açaí', 'df_xsell_SPO_Padaria/Cafeteria/Bolos', 'df_xsell_SPO_Lanches', 'df_xsell_SPO_Prato do dia', 'df_xsell_SPO_Comida brasileira y saudavel', 'df_xsell_SPO_Other', 'df_xsell_SPO_Pizza/Comida Italiana', 'df_xsell_PBC_Other', 'df_xsell_CMX_Other', 'df_xsell_CMX_Panadería/Pastelería', 'df_xsell_CMX_Fonda', 'df_xsell_CMX_Comida rápida', 'df_xsell_CMX_Taquería', 'df_xsell_CMX_Cafetería', 'df_xsell_CMX_Juguería/Comida saludable', 'df_xsell_CMX_Comida especializada', 'df_xsell_BOG_Corrientazo', 'df_xsell_BOG_Panadería/Pastelería', 'df_xsell_BOG_Comida especializada', 'df_xsell_BOG_Empanadas/arepas', 'df_xsell_BOG_Comida rápida', 'df_xsell_BOG_Parrilla/asadero', 'df_xsell_BOG_Other', 'df_xsell_BOG_Cafetería', 'df_xsell_BHZ_Other', 'df_xsell_BAQ_Corrientazo', 'df_xsell_BAQ_Comida especializada', 'df_xsell_BAQ_Other', 'df_xsell_BAQ_Panadería/Pastelería/Cafetería', 'df_xsell_BAQ_Comida rá

In [ ]:
#global_offer_csv.to_csv(f'.csv', index = False)